In [ ]:
import matplotlib.pyplot as plt 
from sklearn import metrics
from sklearn.metrics import confusion_matrix
%matplotlib inline
import seaborn as sns

import numpy as np 
import pandas as pd 

import os
import re
import nltk

In [2]:
df = pd.read_csv('./news.csv')
print(df.shape)
df.head()

(6335, 4)


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
train=pd.read_csv('./news.csv')
test=pd.read_csv('./news.csv')

In [4]:
print(train.shape, test.shape)
train.head()

(6335, 4) (6335, 4)


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
train['title'][0]

'You Can Smell Hillary’s Fear'

In [6]:
test['total']=test['title']+' '+test['text']
train['total']=train['title']+' '+train['text']

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from nltk.corpus import stopwords
nltk.download("stopwords")

stop_words = stopwords.words('english')
print(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [10]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
lemmatizer=WordNetLemmatizer()
for index,row in train.iterrows():
    filter_sentence = ''
    
    sentence = row['total']
    sentence = re.sub(r'[^\w\s]','',sentence)                                                                     #cleaning
    
    words = nltk.word_tokenize(sentence)                                                                        #tokenization
    
    words = [w for w in words if not w in stop_words]                                                       #stopwords removal
    
    for word in words:
        filter_sentence = filter_sentence + ' ' + str(lemmatizer.lemmatize(word)).lower()
        
    train.loc[index,'total'] = filter_sentence

In [12]:
train = train[['total','label']]
train.head(200)

,total,label
0,you can smell hillarys fear daniel greenfield...,FAKE
1,watch the exact moment paul ryan committed po...,FAKE
2,kerry go paris gesture sympathy us secretary ...,REAL
3,bernie supporter twitter erupt anger dnc we t...,FAKE
4,the battle new york why this primary matters ...,REAL
...,...,...
195,raqqamosul politicians fiddle as middle east ...,FAKE
196,chris christie losing homestate donor jeb bus...,REAL
197,washington home depot store receives order fi...,FAKE
198,western lynch mob russia ties itself absurd a...,FAKE


In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [14]:
X_train = train['total']
Y_train = train['label']

In [15]:
def vectorize_text(features, max_features):
    vectorizer = TfidfVectorizer( stop_words='english',
                            decode_error='strict',
                            analyzer='word',
                            ngram_range=(1, 2),
                            max_features=max_features
                            #max_df=0.5 # Verwendet im ML-Kurs unter Preprocessing                   
                            )
    feature_vec = vectorizer.fit_transform(features)
    return feature_vec.toarray()

In [16]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(X_train)
freq_term_matrix = count_vectorizer.transform(X_train)
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(freq_term_matrix)
tf_idf_matrix = tfidf.fit_transform(freq_term_matrix)

In [17]:
tf_idf_matrix

<6335x85109 sparse matrix of type '<class 'numpy.float64'>'
	with 1861649 stored elements in Compressed Sparse Row format>

In [18]:
test_counts = count_vectorizer.transform(test['total'].values)
test_tfidf = tfidf.transform(test_counts)

#split in samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tf_idf_matrix, Y_train, random_state=0)

In [19]:
print(X_train.shape, Y_train.shape)

(4751, 85109) (6335,)


In [20]:
logreg = LogisticRegression(C=1e5)
logreg.fit(X_train, y_train)
pred = logreg.predict(X_test)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))
from sklearn.naive_bayes import MultinomialNB
cm = confusion_matrix(y_test, pred)
cm

Accuracy of Logistic regression classifier on training set: 1.00
Accuracy of Logistic regression classifier on test set: 0.94


array([[719,  48],
       [ 49, 768]], dtype=int64)

In [21]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()
NB.fit(X_train, y_train)
pred = NB.predict(X_test)
print('Accuracy of NB  classifier on training set: {:.2f}'
     .format(NB.score(X_train, y_train)))
print('Accuracy of NB classifier on test set: {:.2f}'
     .format(NB.score(X_test, y_test)))
cm = confusion_matrix(y_test, pred)
cm

Accuracy of NB  classifier on training set: 0.90
Accuracy of NB classifier on test set: 0.82


array([[497, 270],
       [ 10, 807]], dtype=int64)

In [22]:
X_train = train['total']
Y_train = train['label']

In [23]:
from sklearn.pipeline import Pipeline
import joblib
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('clf', linear_model.LogisticRegression(C=1e5)),
])

In [25]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=100000.0))])

In [26]:
print(X_train[0])

 you can smell hillarys fear daniel greenfield shillman journalism fellow freedom center new york writer focusing radical islam in final stretch election hillary rodham clinton gone war fbi the word unprecedented thrown around often election ought retired but still unprecedented nominee major political party go war fbi but thats exactly hillary people done coma patient waking watching hour cnn hospital bed would assume fbi director james comey hillarys opponent election the fbi attack everyone obama cnn hillarys people circulated letter attacking comey there currently medium hit piece lambasting targeting trump it wouldnt surprising clintons ally start running attack ad fbi the fbis leadership warned entire leftwing establishment form lynch mob continue going hillary and fbis credibility attacked medium democrats preemptively head result investigation clinton foundation hillary clinton the covert struggle fbi agent obamas doj people gone explosively public the new york times compared c

In [35]:
pipeline.predict(["Burundi's longtime President Pierre Nkurunziza has congratulated the governing party's hand-picked successor on a large victory in the country's presidential election, though the main opposition has pledged to contest the result in court. The election commission on Monday declared Evariste Ndayishimiye, a former army general chosen by the CNDD-FDD party as heir to Nkurunziza, the winner of the May 20 poll with 68.72 percent vote."])

array(['FAKE'], dtype=object)

In [36]:
filename = 'pipeline.sav'
joblib.dump(pipeline, filename)

['pipeline.sav']

In [37]:
filename = './App/pipeline.sav'

In [38]:
loaded_model = joblib.load(filename)
result = loaded_model.predict(["The nationwide tally of coronavirus cases have crossed the 1,45,000-mark while the death toll has surged past 4,100, as per the latest information available at the Ministry of Health and Family Welfare. According to the Health Ministry data, there are 80,722 active coronavirus cases in India while 60,490 have been cured and discharged from hospitals. Amid this, the Indian Council of Medical Research has said that the top 10 hotspot cities reporting the highest number of COVID-19 cases in the country will be covered in a serosurvey to estimate the prevalence of community transmission of the infection.The survey would also be conducted in 60 districts of 21 states and Union Territories categorized into four strata -- zero, low, medium and high -- according to the reported COVID cases per million population, as per the protocols for community-based surveillance published in the IJMR."])
print(result) 

['FAKE']
